In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai import *
from fastai.tabular import *
from fastai.tabular.models import TabularModel

In [ ]:
PATH = Path('../data/rossmann/')
train_df = pd.read_feather(PATH/'train_clean')
test_df = pd.read_feather(PATH/'test_clean')

In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

cont_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

In [ ]:
dep_var = 'Sales'
train_df = pd.read_feather(PATH/'train_clean')
train_df = train_df[cat_vars+cont_vars+[dep_var, 'Date']].copy()

In [ ]:
test_df['Date'].min(), test_df['Date'].max(), len(test_df)

In [ ]:
cut = train_df['Date'][(train_df['Date'] == train_df['Date'][len(test_df)])].index.max()
cut

In [ ]:
train_df = train_df.set_index('Date')
train_df,valid_df = train_df[cut:], train_df[:cut]
len(train_df),len(valid_df)

In [ ]:
tfms = [FillMissing, Categorify]
data = tabular_data_from_df(PATH, train_df, valid_df, dep_var, tfms=[FillMissing, Categorify], cat_names=cat_vars, 
                           cont_names=cont_vars, log_output=True, num_workers=0)

In [ ]:
cat_szs = [len(train_df[n].cat.categories)+1 for n in cat_vars]
emb_szs = [(c, min(50, (c+1)//2)) for c in cat_szs]
emb_szs

In [ ]:
max_log_y = np.log(np.max(train_df['Sales']))
y_range = torch.tensor([0, max_log_y*1.2], device=default_device)

In [ ]:
model = TabularModel(emb_szs, len(cont_vars), 1, [1000,500], [0.001,0.01], emb_drop=0.04, y_range=y_range, is_reg=True)

In [ ]:
learn = Learner(data, model)
learn.loss_fn = F.mse_loss
learn.metrics = [exp_rmspe]

In [ ]:
learn.fit_one_cycle(1, 1e-3, wd=0.2, pct_start=0.2)